## This Notebook is to explore and cluster the neighborhoods in Toronto

In [1]:
# Importing Necessary Libraries
import numpy as np
import pandas as pd
import json
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium 
from sklearn.cluster import KMeans

In [2]:
# Importing Toronto Neighborhoods Dataframe 
tor = pd.read_csv("Toronto_neighborhoods.csv")
print(tor.shape)
tor.head()

(103, 3)


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [ ]:
# Iterate to get co-ordinates of the Toronto neighborhoods 

import geocoder

for i, pc in enumerate(tor['PostalCode']):
    
    postal_code = i
    lat_lng_coords = None

    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    
    tor['Latitude'][i] = lat_lng_coords[0]
    tor['Longitude'][i] = lat_lng_coords[1]

tor.head()

The above loop takes too long to fetch the co-ordinates, hence going ahead with the readily provided co-ordinates data

In [4]:
# Importing Toronto Neighborhood Co-ordinares Dataframe 
coord = pd.read_csv("Geospatial_Coordinates.csv")
coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [5]:
# A loop to ensure that co-ordinates from coord dataframe are matched rightly postal-code wise, 
# One can also sort both the dataframes according to Postal Codes and then concatenate
latitude = []
longitude = []
for i, torcod in enumerate(tor['PostalCode']):
    for j, coordcod in enumerate(coord['Postal Code']):
        if i == j:
            latitude.append(coord['Latitude'][j])
            longitude.append(coord['Longitude'][j])
            break
tor['Latitude'] = pd.Series(latitude)
tor['Longitude'] = pd.Series(longitude)
tor.loc[tor['PostalCode']=='M5G'] # Random check to make sure the co-ordinates are correct

,PostalCode,Borough,Neighborhood,Latitude,Longitude
57,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
